# Reading data

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")
from pandas.tseries.offsets import DateOffset
from collections import Counter

In [2]:
df = pd.read_csv('data/dataset_fixed.csv')
sample_submission = pd.read_csv('data/submission_example.csv')

In [3]:
# Необходимые пороги
start_train_threshold = '2020-03-01' # Должно быть хотя бы одно привлечение позже этой даты, иначе партнер уже "ушел"
train_threshold = '2020-06-01' # Конец трейна (привлечения позже этой даты не подаем в X_train)
val_threshold = '2020-09-01' # Конец валидации (привлечения позже этой даты не подаем в X_val)
test_threshold = '2020-12-01' # Конец теста (привлечения позже этой даты не подаем в X_test)

In [4]:
df['time'] = pd.to_datetime(df['time'])
df['start_time'] = pd.to_datetime(df['start_time'])
df

,partner,client,type,time,start_time
0,122027,5579,3,2019-06-25,2019-02-01
1,270277,5585,3,2020-05-07,2020-04-01
2,238679,5586,3,2020-02-27,2020-02-01
3,118398,5587,3,2020-03-26,2019-09-01
4,10402,5588,0,2019-04-05,2019-01-01
...,...,...,...,...,...
130813,4639,333509,4,2020-07-20,2020-03-01
130814,226754,333510,3,2020-07-19,2020-05-01
130815,2645,333511,4,2020-07-23,2020-05-01
130816,145720,333512,0,2020-07-25,2019-05-01


In [5]:
# Нарезаем датасет на train/train_full/val/test
# Train full - полная выборка, нужна для обучения без валидации (это осуществяется непосредственно перед инференсом)

X_train = df[df.time < train_threshold] # 2019-03-01 - 2020-03-01
X_val = df[(df.time.min() + DateOffset(months=3) < df.time) & (df.time < val_threshold)] # 2019-06-01 - 2020-06-01

X_train_full = df[df.time < val_threshold] # 2019-03-01 - 2020-09-01

X_test = df[df.time < test_threshold] # 2019-03-01 - 2020-12-01
X_test = X_test[X_test.time > df.time.min() + DateOffset(months=3)] # 2019-06-01 - 2020-12-01

# Feature engineering

In [6]:
# Сколько партнеров пришли в этот день
partners_by_start_time = X_train.drop_duplicates('partner')['start_time'].value_counts()

In [7]:
def num_unique_values(x):
    # Число уникальных значений фичи для партнера
    return len(set(x))

def freq(x):
    # Самое частое значение фичи для партнера
    return Counter(x).most_common(1)[0][0]


def get_fixed(x):
    # Получаем фиксированное значение (например, тип партнера не зависит от времени, тут для партнера мы получим одно число - его тип)
    return x.iloc[0]


def get_last_n_and_pad(x, n):
    # Получаем время до порога для n последних привлечений. Делаем паддинг (если меньше 5 привлечений у партнера)
    x = x[-n:]

    if isinstance(x, int):
        x = [x]
    else:
        x = x.tolist()

    return (n - len(x)) * [-1] + x


def data_to_last_actions(data):
    # Получаем время до порога для n последних привлечений в нужном формате.
    
    n = 5
    group_time = data.groupby('partner')['time_left'].agg(lambda x: get_last_n_and_pad(x, n))
    group_time = pd.DataFrame(group_time)
    group_time.columns=['times']
    group_time[[f'time_left_{i}_attraction' for i in range(n)]] = group_time['times'].apply(lambda x: pd.Series(x)).values
    group_time.drop(columns=['times'], inplace=True)


    data = data.drop_duplicates(subset=['partner'])
    data = data.merge(group_time, left_on = 'partner', right_index=True).drop(columns=['time_left'])
    return data

def get_diffs(x):
    # Сколько времени с предыдущего привлечения до текущего прошло
    x = x.values.tolist()
    x = [np.nan] + (np.array(sorted(x, reverse=True)[:-1]) - np.array(sorted(x, reverse=True)[1:])).tolist()

    return x
    
def get_diff_more_than_90_count(x):
    # Сколько у партнера было 'перерывов' в привлечениях дольше 90 дней
    return len(x[x>90])


def get_features(data, target, last_attraction_threshold, threshold):
    """
    Добавляет фичи в данные, аггрерирует их
    :param data: pd.DataFrame (DataFrame с привлечениями)
    :param target: pd.DataFrame (DataFrame с таргетом, для тестовой выборки None)
    :param last_attraction_threshold: str/pd.DateTime (хоть одно привлечение должно быть позже этой даты,
    иначе считаем партнера уже ушедшим)
    :param threshold: str/pd.DateTime (берем привлечения до этой даты)
    :return: pd.DataFrame, pd.DataFrame - фичи и таргет (для теста только фичи)
    """
    
    
    aggs = {
        "start_time": get_fixed,
        "type": [get_fixed],
        "time_left": ['mean', 'median', 'sum', 'std', 'min', 'max'],
        "time_days": ['mean', 'median', 'min', 'max'],
        'start_time': get_fixed,
        'day_of_week': [freq, num_unique_values],
        'month': [freq, num_unique_values],
        'year': [freq, num_unique_values],
        'diff': ['mean', 'median', 'std', get_diff_more_than_90_count, 'min', 'max', 'sum'] # num_unique_values
        # 'client': ['median', 'min', 'max']
        }
    
    
    # Оставляем только не ушедших партнеров
    max_time_of_attraction = data.groupby('partner').agg({'time': 'max'})
    appropriate_partners = max_time_of_attraction[max_time_of_attraction.time>=last_attraction_threshold].index
    data = data[data.partner.isin(appropriate_partners)]
    
    
    # Time
    data['time'] = pd.to_datetime(data['time'])
    data['time_left'] = (pd.to_datetime(threshold) - data['time']).astype(int)//10**9 / 3600 // 24
    data['time_days'] = data['time'].astype(int)//10**9 / 3600 // 24
    data['day_of_week'] = data['time'].dt.day_of_week
    data['month'] = data['time'].dt.month
    data['year'] = data['time'].dt.year
    
    data.drop(columns=['time'], inplace=True)
    

    
    data.sort_values(['partner', 'time_left'], inplace=True)
    data['diff'] = data.groupby('partner')['time_left'].transform(get_diffs)
    
    
    cols_to_agg = list(aggs.keys())
    
    # Начинаем аггрегацию данных
    data_agg = data[['partner', cols_to_agg[0]]].groupby("partner", as_index=True).agg({cols_to_agg[0]: aggs[cols_to_agg[0]]})
    for feat in cols_to_agg[1:]:
        data_agg = pd.concat([data_agg, data[['partner', feat]].groupby("partner", as_index=True).agg({feat: aggs[feat]})], axis=1)
        
    
    # Исправляем названия столбцов
    data_agg.columns = data_agg.columns.values.tolist()[:1] +  ['_'.join(col).strip('_') for col in data_agg.columns.values[1:]]
    data_agg.columns = [col if 'get_fixed' not in col else col[:-10] for col in data_agg.columns]
    if 'diff_get_diff_more_than_90_count' in data_agg:
        data_agg.rename(columns={'diff_get_diff_more_than_90_count': 'diff_more_than_90'}, inplace=True)
        
    
    data_agg.insert(2, 'count', data.groupby('partner').size()) # Число привлечений
    
    
    # Заполняем наны для партнеров с одним привлечением
    std_cols = [col for col in data_agg.columns if 'std' in col]
    data_agg.loc[:, std_cols] = data_agg.loc[:, std_cols].fillna(-1)
    # data_agg.fillna(0, inplace=True)
    
    
    # Start time
    # data_agg['partners_by_start_time'] = data_agg['start_time'].map(partners_by_start_time.to_dict()).fillna(1)
    data_agg['start_time'] = pd.to_datetime(data_agg['start_time'])
    data_agg['start_time_left'] = (pd.to_datetime(threshold) - data_agg['start_time']).astype(int)//10**9 / 3600 // 24
    data_agg['start_time_days'] = data_agg['start_time'].astype(int)//10**9 / 3600 // 24
    data_agg['day_of_week_start'] =data_agg['start_time'].dt.day_of_week
    data_agg['month_start'] = data_agg['start_time'].dt.month
    data_agg['year_start'] = data_agg['start_time'].dt.year
    
    data_agg['start_time'] = data_agg['start_time'].astype(int)//10**9//3600//24
    # data_agg.drop(columns=['start_time'], inplace=True)
    

    
    
    if target is None:
        data_agg = data_agg.loc[sample_submission['clientbankpartner_pin'].values]
    
    
    
    # last_actions_data = data_to_last_actions(data.copy()[['partner', 'time_left']]).set_index('partner')
    # for col in last_actions_data:
    #     if col not in data_agg:
    #         data_agg[col] = last_actions_data[col].values

    
    
    # Для каждого уникального значения категориальной фичи считаем число привлечений и долю привлечений (например, сколько привлечений в марте было)
    # categorial = ['day_of_week', 'month', 'year']
    # for cat in categorial:
    #     catf = data.groupby(['partner', cat]).size()
    #     catf = catf.unstack(fill_value=0.0)
    #     columns = catf.columns.values
    #     catf.columns = [f"{cat}_{col}_count" for col in columns]
    #     data_agg = data_agg.merge(catf, left_index=True, right_on="partner")

    #     catf = catf.div(catf.sum(axis=1), axis=0)
    #     catf.columns = [f"{cat}_{col}_part" for col in columns]
    #     data_agg = data_agg.merge(catf, left_index=True, right_on="partner")
    #     del catf
        
    
    cat_cols = ['type']
    num_cols = [col for col in data_agg.columns if col not in cat_cols+['score']]
    
    data_agg[cat_cols] = data_agg[cat_cols].astype('int')
    data_agg[num_cols] = data_agg[num_cols].astype('float')
    

    if target is not None:
        data_agg = data_agg.join(target.set_index("partner"), "partner")
        return data_agg.drop(columns=["score"]), data_agg[["score"]]
    else:
        return data_agg
    

# Train and Train_full

Создадим Train и Full train с аугментацией (нарезая датасет на части).

Пример:

Партиция 0: данные с 2019-03-01 по 2020-02-28, таргет с 2020-03-01 по 2020-05-31

Партиция 1: данные с 2019-02-15 по 2020-02-13, таргет с 2020-02-15 по 2020-05-16

Партиция 2: данные с 2019-02-01 по 2020-01-29, таргет с 2020-02-01 по 2020-05-01

...

In [8]:
delta = DateOffset(days=15) # С каким промежутком нарезать
num_partitions = 26 # Сколько партиций

In [9]:
X_train_list = []
y_train_list = []


for num_partition in range(num_partitions):
    # Пороги для партиции
    start_train_threshold_new = pd.to_datetime(start_train_threshold) - delta * num_partition
    train_threshold_new = pd.to_datetime(train_threshold) - delta * num_partition
    val_threshold_new = pd.to_datetime(val_threshold) - delta * num_partition
    
    
    # Данные для одной партиции
    data = X_train[X_train.time < train_threshold_new]
    y_train_part = pd.DataFrame({'partner': np.unique(data.partner), 'score': 1})
    y_train_part.loc[y_train_part.partner.isin(np.unique(df[(train_threshold_new <= df.time) & ( df.time < val_threshold_new)]['partner'])), 'score'] = 0

    X_train_part, y_train_part = get_features(data.copy(), y_train_part, start_train_threshold_new, train_threshold_new)
    X_train_part['partition'] = num_partition
    
    
    
    X_train_list.append(X_train_part)
    y_train_list.append(y_train_part)
    # print(X_train_part.shape)
    # print(y_train_part.score.value_counts().to_dict())
    
# Объединяем партиции в единый датасет
X_train = pd.concat(X_train_list, axis=0, ignore_index=True)
y_train = pd.concat(y_train_list, axis=0, ignore_index=True)

In [10]:
X_train_list = []
y_train_list = []


for num_partition in range(num_partitions):
    # Пороги для партиции
    start_train_threshold_new = pd.to_datetime(train_threshold) - delta * num_partition
    train_threshold_new = pd.to_datetime(val_threshold) - delta * num_partition
    val_threshold_new = pd.to_datetime(test_threshold) - delta * num_partition
    
    
    # Данные для одной партиции
    data = X_train_full[X_train_full.time < train_threshold_new]
    y_train_part = pd.DataFrame({'partner': np.unique(data.partner), 'score': 1})
    y_train_part.loc[y_train_part.partner.isin(np.unique(df[(train_threshold_new <= df.time) & ( df.time < val_threshold_new)]['partner'])), 'score'] = 0
    
    X_train_part, y_train_part = get_features(data.copy(), y_train_part, start_train_threshold_new, train_threshold_new)
    X_train_part['partition'] = num_partition
    
    
    X_train_list.append(X_train_part)
    y_train_list.append(y_train_part)
    # print(X_train_part.shape)
    # print(y_train_part.score.value_counts().to_dict())

# Объединяем партиции в единый датасет
X_train_full = pd.concat(X_train_list, axis=0, ignore_index=True)
y_train_full = pd.concat(y_train_list, axis=0, ignore_index=True)

# Val and Test

In [11]:
y_val = pd.DataFrame({'partner': np.unique(X_val.partner), 'score': 1})
y_val.loc[y_val.partner.isin(np.unique(df[df.time >= val_threshold]['partner'])), 'score'] = 0
print(y_val.score.value_counts())
y_val

score
1    4616
0    2914
Name: count, dtype: int64


,partner,score
0,1,0
1,2,0
2,5,1
3,6,1
4,9,1
...,...,...
7525,333400,0
7526,333459,0
7527,333492,0
7528,333511,1


In [12]:
X_val, y_val = get_features(X_val, y_val, train_threshold, val_threshold)
X_test = get_features(X_test, None, val_threshold, test_threshold)

# Saving data

In [13]:
X_train.shape

(65114, 32)

In [14]:
X_train.head(3)

,start_time,type,count,time_left_mean,time_left_median,time_left_sum,time_left_std,time_left_min,time_left_max,time_days_mean,...,diff_more_than_90,diff_min,diff_max,diff_sum,start_time_left,start_time_days,day_of_week_start,month_start,year_start,partition
0,18109.0,4,2.0,96.5,96.5,193.0,132.228968,3.0,190.0,18317.5,...,1.0,187.0,187.0,187.0,305.0,18109.0,3.0,8.0,2019.0,0
1,18201.0,4,7.0,131.0,140.0,917.0,40.636601,50.0,177.0,18283.0,...,0.0,0.0,68.0,127.0,213.0,18201.0,4.0,11.0,2019.0,0
2,17866.0,4,18.0,233.5,239.5,4203.0,124.242765,10.0,400.0,18180.5,...,0.0,0.0,61.0,390.0,548.0,17866.0,5.0,12.0,2018.0,0


In [15]:
X_test.head(3)

,start_time,type,count,time_left_mean,time_left_median,time_left_sum,time_left_std,time_left_min,time_left_max,time_days_mean,...,diff_std,diff_more_than_90,diff_min,diff_max,diff_sum,start_time_left,start_time_days,day_of_week_start,month_start,year_start
partner,,,,,,,,,,,,,,,,,,,,,
6781,18436.0,3,145.0,32.455172,34.0,4706.0,21.659928,1.0,113.0,18564.544828,...,2.767050,0.0,0.0,31.0,112.0,161.0,18436.0,1.0,6.0,2020.0
236905,18322.0,3,29.0,123.034483,109.0,3568.0,66.341370,6.0,240.0,18473.965517,...,10.435744,0.0,0.0,45.0,234.0,275.0,18322.0,6.0,3.0,2020.0
125779,18231.0,3,10.0,206.800000,251.0,2068.0,104.637788,12.0,344.0,18390.200000,...,38.452713,1.0,0.0,91.0,332.0,366.0,18231.0,6.0,12.0,2019.0


In [16]:
X_train.to_parquet('data/X_train_processed.parquet')
X_train_full.to_parquet('data/X_train_full_processed.parquet')
X_val.to_parquet('data/X_val_processed.parquet')
X_test.to_parquet('data/X_test_processed.parquet')
y_train.to_parquet('data/y_train_processed.parquet')
y_train_full.to_parquet('data/y_train_full_processed.parquet')
y_val.to_parquet('data/y_val_processed.parquet')

CPU times: user 125 ms, sys: 31.4 ms, total: 156 ms
Wall time: 130 ms


In [17]:
X_train.describe()

,start_time,type,count,time_left_mean,time_left_median,time_left_sum,time_left_std,time_left_min,time_left_max,time_days_mean,...,diff_more_than_90,diff_min,diff_max,diff_sum,start_time_left,start_time_days,day_of_week_start,month_start,year_start,partition
count,65114.000000,65114.000000,65114.000000,65114.000000,65114.000000,65114.000000,65114.000000,65114.000000,65114.000000,65114.000000,...,65114.000000,49887.000000,49887.000000,65114.000000,65114.000000,65114.000000,65114.000000,65114.000000,65114.000000,65114.000000
mean,18083.854701,2.850339,11.545044,78.656481,79.612280,1286.828148,37.366696,29.829775,125.484043,18179.256579,...,0.116350,14.693106,53.704833,95.654268,174.058359,18083.854701,3.135716,6.082947,2019.090334,10.405796
std,132.699238,1.272089,31.315250,57.291780,62.536603,4599.369514,38.869404,25.314711,102.513186,105.845879,...,0.342149,30.952520,48.133052,103.324689,115.854373,132.699238,2.027030,3.567789,0.430237,7.212758
min,17866.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-1.000000,1.000000,1.000000,17956.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,17866.000000,0.000000,1.000000,2018.000000,0.000000
25%,17987.000000,3.000000,2.000000,35.000000,33.500000,65.000000,0.707107,8.000000,47.000000,18094.513889,...,0.000000,0.000000,22.000000,1.000000,81.000000,17987.000000,1.000000,3.000000,2019.000000,4.000000
50%,18078.000000,3.000000,4.000000,67.000000,65.500000,233.000000,28.284271,22.000000,93.000000,18175.000000,...,0.000000,2.000000,42.000000,62.000000,142.000000,18078.000000,4.000000,6.000000,2019.000000,10.000000
75%,18170.000000,4.000000,9.000000,108.200000,108.000000,863.000000,59.059292,47.000000,183.000000,18262.000000,...,0.000000,18.000000,70.000000,154.000000,245.000000,18170.000000,5.000000,9.000000,2019.000000,16.000000
max,18383.000000,4.000000,906.000000,381.879518,439.000000,193528.000000,277.185858,92.000000,458.000000,18412.000000,...,3.000000,392.000000,405.000000,456.000000,548.000000,18383.000000,6.000000,12.000000,2020.000000,25.000000
